In [1]:
# Clone PyFM
!git clone https://github.com/RobinMagnet/pyFM.git pyfm_repo
!mv pyfm_repo/pyFM .
!rm -rf pyfm_repo

Cloning into 'pyfm_repo'...
remote: Enumerating objects: 512, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 512 (delta 59), reused 56 (delta 50), pack-reused 436
Receiving objects: 100% (512/512), 942.38 KiB | 8.89 MiB/s, done.
Resolving deltas: 100% (324/324), done.
mv: rename pyfm_repo/pyFM to ./pyFM: Directory not empty



# 1 - Imports and defining functions

In [1]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 2 - Computing the functional map

**Loading data**

In [101]:
num=4
mesh1 = TriMesh('data/dataset/YellowToy01/yellow_push_toy_1_100000.obj')
mesh2 = TriMesh("data/dataset/YellowToy01/yellow_push_toy_"+str(num)+"_50000.obj")
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

double_plot(mesh1,mesh2)

Mesh 1 : 50002 vertices, 100000 faces
Mesh 2 : 25002 vertices, 50000 faces


In [102]:
from scipy.spatial import distance
import numpy as np
rand_idx=np.random.permutation(mesh1.vertlist.shape[0])[:20]
print(rand_idx)
euc_dist=distance.cdist(mesh1.vertlist[rand_idx],mesh2.vertlist)

[23276 21871 20294 49737 49116 13416 11530 14285 44790  4558 12183 44546
 33063 41545 38128 46895  6587 23063 15499 48590]


In [103]:
temp=np.argmin(euc_dist, axis=1)

In [104]:
#rand_idx=np.random.randint(temp.shape[0], size=20)

In [105]:
# Manually change the landmark file name
with open("data/yellow_toy_1-"+str(num)+"_landmarks.txt", 'w+') as fp:
      
    for i in range(temp.shape[0]):
        fp.write(str(rand_idx[i])+" "+str(temp[i])+"\n")


**Computing descriptors**

In [106]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt("data/yellow_toy_1-"+str(num)+"_landmarks.txt",dtype=int),  # loading 10 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 22.43 s
Computing 200 eigenvectors
	Done in 8.90 s

Computing descriptors
	Normalizing descriptors

	420 out of 2100 possible descriptors kept


**Fitting the model**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [107]:
fit_params = {
    'w_descr': 1e0,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}



model.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 1.0e-11

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 420 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 25, nit : 22, warnflag : 0
	Done in 0.99 seconds


**Visualizing the associated point to point map**

In [108]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 3 - Refining the Functional Map
```model.FM``` returns the current state of functional map. One can change which one is returned by using ```model.change_FM_type(FM_type)```, as one can see below. 

**ICP**

In [109]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/10 [00:00<?, ?it/s]

**Zoomout**

In [110]:
model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one
model.zoomout_refine(nit=15, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/15 [00:00<?, ?it/s]

(50, 50)


In [111]:
print(mesh1.vertlist[p2p_21_zo].shape,mesh2.vertlist.shape)
np.save("Correspondences/YellowToy01/not_deformed_"+str(num)+"_correspondences.npy",mesh1.vertlist[p2p_21_zo])
np.save("Correspondences/YellowToy01/deformed_"+str(num)+"_correspondences.npy",mesh2.vertlist )

(25002, 3) (25002, 3)


In [ ]:
# for 1-3: 30 landmarks and 10 sub_samples were used